In [93]:
reload_lamb()

In [94]:
%%lamb
card = Card_<e,n>
maxfun = Max_<{n},n>
||many|| = L d_n : L f_<e,t> : L x_e : f(x) & (card(x) >= d)

${card}_{\langle{}e,n\rangle{}}\:=\:{Card}_{\langle{}e,n\rangle{}}$<br />
${maxfun}_{\langle{}\{n\},n\rangle{}}\:=\:{Max}_{\langle{}\{n\},n\rangle{}}$<br />
$||\mathbf{\text{many}}||^{}_{\langle{}n,\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}\rangle{}} \:=\: \lambda{} d_{n} \: . \: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \lambda{} x_{e} \: . \: ({f}_{\langle{}e,t\rangle{}}({x}_{e}) \wedge{} ({d}_{n} \leq{} {Card}_{\langle{}e,n\rangle{}}({x}_{e})))$

In [95]:
%%lamb
dset = L d_<n,<e,t>> : L x_e : Set d2_n: d(d2)(x)
#||est|| = L c_{e} : L d_<n,<e,t>> : L x_e : Forall y_e: (c << y) >> ((y != x) >>  (maxfun(dset(d)(x)) > maxfun(dset(d)(y))))

${dset}_{\langle{}\langle{}n,\langle{}e,t\rangle{}\rangle{},\langle{}e,\{n\}\rangle{}\rangle{}}\:=\:\lambda{} d_{\langle{}n,\langle{}e,t\rangle{}\rangle{}} \: . \: \lambda{} x_{e} \: . \: \{d2_{n}\:|\: {d}_{\langle{}n,\langle{}e,t\rangle{}\rangle{}}({d2}_{n})({x}_{e})\}$

In [96]:
maxfun(dset("d_<n,<e,t>>")("x_e")).reduce_all()

Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(x_e)))

In [97]:
%%lamb
||est|| = L c_{e} : L d_<n,<e,t>> : L x_e : Forall y_e: (y << c) >> ((~(y <=> x)) >> (maxfun(dset(d)(x)) > maxfun(dset(d)(y))))
est.reduce_all()        

$||\mathbf{\text{est}}||^{}_{\langle{}\{e\},\langle{}\langle{}n,\langle{}e,t\rangle{}\rangle{},\langle{}e,t\rangle{}\rangle{}\rangle{}} \:=\: \lambda{} c_{\{e\}} \: . \: \lambda{} d_{\langle{}n,\langle{}e,t\rangle{}\rangle{}} \: . \: \lambda{} x_{e} \: . \: \forall{} y_{e} \: . \: (({y}_{e} \in{} {c}_{\{e\}}) \rightarrow{} (\neg{} ({y}_{e} = {x}_{e}) \rightarrow{} ({Max}_{\langle{}\{n\},n\rangle{}}(\{d2_{n}\:|\: {d}_{\langle{}n,\langle{}e,t\rangle{}\rangle{}}({d2}_{n})({x}_{e})\}) > {Max}_{\langle{}\{n\},n\rangle{}}(\{d2_{n}\:|\: {d}_{\langle{}n,\langle{}e,t\rangle{}\rangle{}}({d2}_{n})({y}_{e})\}))))$

In [98]:
te("y_e << c_{e}")

(y_e ∈ c_{e})

In [99]:
te("~(y_e <=> x_e)")

~(y_e == x_e)

In [100]:
%%lamb
||high|| = L d_n : L x_e : Height_<e,n>(x) >= d
||mountain|| = L x_e : Mt(x)
||climbed|| = L x_e : L y_e : Climbed(y,x)
||the|| = L f_<e,t> : Iota x_e : f(x)
||C|| = C_{e}

INFO (meta): Coerced guessed type t for 'Mt_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type t for 'Climbed_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$||\mathbf{\text{high}}||^{}_{\langle{}n,\langle{}e,t\rangle{}\rangle{}} \:=\: \lambda{} d_{n} \: . \: \lambda{} x_{e} \: . \: ({d}_{n} \leq{} {Height}_{\langle{}e,n\rangle{}}({x}_{e}))$<br />
$||\mathbf{\text{mountain}}||^{}_{\langle{}e,t\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Mt}({x}_{e})$<br />
$||\mathbf{\text{climbed}}||^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: \lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Climbed}({y}_{e}, {x}_{e})$<br />
$||\mathbf{\text{the}}||^{}_{\langle{}\langle{}e,t\rangle{},e\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \iota{} x_{e} \: . \: {f}_{\langle{}e,t\rangle{}}({x}_{e})$<br />
$||\mathbf{\text{C}}||^{}_{\{e\}} \:=\: {C}_{\{e\}}$

In [101]:
trace = lang.Trace(index=2, typ=tp("n"))
trace

||t2|| = t2_n

In [102]:
def pa_fun_n(binder, content, assignment=None):
    if (binder.content is not None) or not binder.name.strip().isnumeric():
        raise types.TypeMismatch(binder, content, "Predicate Abstraction")
    index = int(binder.name.strip())
    vname = "t%i" % index
    # there could be more natural ways to do this...should H&K assignment functions be implemented directly?
    outer_vname = "x"
    inner_fun = meta.LFun(types.type_n, content.content.under_assignment(assignment), vname)
    used_variables = inner_fun.free_variables() | inner_fun.bound_variables() # totally brute force...
    outer_vname = meta.alpha_variant(outer_vname, used_variables)
    f = meta.LFun(types.type_n, (inner_fun)(outer_vname + "_n").reduce(), outer_vname)
    return lang.BinaryComposite(binder, content, f)

system = lang.td_system.copy()
system.add_rule(lang.BinaryCompositionOp("PA-n", pa_fun_n, allow_none=True))
lang.set_system(system)
lang.get_system()

Composition system: H&K simple (copy)

In [103]:
binder = lang.Binder(index=2)
binder

||2|| = None

In [104]:
(est * C) * (binder * ((trace * high) * mountain))

CompositionResult(results=[||[[est C] [2 [[high t2] mountain]]]|| = (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: ((d2_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e)))) > Max_<{n},n>((Set d2_n: ((d2_n <= Height_<e,n>(y_e)) & Mt_<e,t>(y_e)))))))))], failures=[||[[2 [[high t2] mountain]] [est C]]|| = Type mismatch: '||[2 [[high t2] mountain]]|| = (λ x1_n: (λ x_e: ((x1_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e))))'/<n,<e,t>> and '||[est C]|| = (λ d_<n,<e,t>>: (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(x_e))) > Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(y_e)))))))))'/<<n,<e,t>>,<e,t>> conflict (mode: Function Application), ||[[est C] [2 [[high t2] mountain]]]|| = Type mismatch: '||[est C]|| = (λ d_<n,<e,t>>: (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(x_e))) > Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(y_e)))))))))'/<<n,<e,t>>,<e,t>> and '||[2 [[high t2] mount

In [105]:
(est * C) * (binder * (climbed * (the * ((trace * high) * mountain))))

CompositionResult(results=[||[[est C] [2 [climbed [the [[high t2] mountain]]]]]|| = (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: Climbed_<(e,e),t>(x_e, (ι x_e: ((d2_n <= Height_<e,n>(x1_e)) & Mt_<e,t>(x1_e)))))) > Max_<{n},n>((Set d2_n: Climbed_<(e,e),t>(y_e, (ι x_e: ((d2_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e)))))))))))], failures=[||[[2 [climbed [the [[high t2] mountain]]]] [est C]]|| = Type mismatch: '||[2 [climbed [the [[high t2] mountain]]]]|| = (λ x1_n: (λ y_e: Climbed_<(e,e),t>(y_e, (ι x_e: ((x1_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e))))))'/<n,<e,t>> and '||[est C]|| = (λ d_<n,<e,t>>: (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(x_e))) > Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(y_e)))))))))'/<<n,<e,t>>,<e,t>> conflict (mode: Function Application), ||[[est C] [2 [climbed [the [[high t2] mountain]]]]]|| = Type mismatch: '||[est C]|| = (λ d_<n,<e,t>>: (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(

In [106]:
%%lamb
||a2|| = L f_<e,t> : L g_<e,<e,t>> : L y_e : Exists x_e : f(x) & g(x)(y) # type-lifted 'a'

$||\mathbf{\text{a2}}||^{}_{\langle{}\langle{}e,t\rangle{},\langle{}\langle{}e,\langle{}e,t\rangle{}\rangle{},\langle{}e,t\rangle{}\rangle{}\rangle{}} \:=\: \lambda{} f_{\langle{}e,t\rangle{}} \: . \: \lambda{} g_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \: . \: \lambda{} y_{e} \: . \: \exists{} x_{e} \: . \: ({f}_{\langle{}e,t\rangle{}}({x}_{e}) \wedge{} {g}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({x}_{e})({y}_{e}))$

In [107]:
(est * C) * (binder * (climbed * (a2 * ((trace * high) * mountain))))

CompositionResult(results=[||[[est C] [2 [[a2 [[high t2] mountain]] climbed]]]|| = (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: (Exists x1_e: (((d2_n <= Height_<e,n>(x1_e)) & Mt_<e,t>(x1_e)) & Climbed_<(e,e),t>(x_e, x1_e))))) > Max_<{n},n>((Set d2_n: (Exists x_e: (((d2_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e)) & Climbed_<(e,e),t>(y_e, x_e))))))))))], failures=[||[[2 [[a2 [[high t2] mountain]] climbed]] [est C]]|| = Type mismatch: '||[2 [[a2 [[high t2] mountain]] climbed]]|| = (λ x1_n: (λ y_e: (Exists x_e: (((x1_n <= Height_<e,n>(x_e)) & Mt_<e,t>(x_e)) & Climbed_<(e,e),t>(y_e, x_e)))))'/<n,<e,t>> and '||[est C]|| = (λ d_<n,<e,t>>: (λ x_e: (Forall y_e: ((y_e ∈ C_{e}) >> (~(y_e == x_e) >> (Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(x_e))) > Max_<{n},n>((Set d2_n: d_<n,<e,t>>(d2_n)(y_e)))))))))'/<<n,<e,t>>,<e,t>> conflict (mode: Function Application), ||[[est C] [2 [[a2 [[high t2] mountain]] climbed]]]|| = Type mismatch: '||[est C]|| = (λ d_<n,<e,t>>: (λ x_